In [1]:
from Crypto.Cipher import AES
import filetype
from Crypto.PublicKey import RSA
from glob import glob
from math import gcd
from sympy import *
import os

In [2]:
ruta = "./keys/"

In [3]:
aes = ruta + "miquel.umbert.bosch_AES_pseudo.enc"

In [4]:
pem = ruta + "miquel.umbert.bosch" + "_pubkeyRSA_pseudo.pem"

In [5]:
rsa = ruta + "miquel.umbert.bosch_RSA_pseudo.enc"

In [6]:
with open(pem, 'r') as f:
    rsakey = f.read()

pubkey = RSA.importKey(rsakey)
personal = pubkey.n
public_exponent = pubkey.e
print(personal)
print()
print(public_exponent)

21480175498539751428436914447616157245398064218512269363091426475010720513575067096409624123574417919917281058081314520520081813039462015597551240691757649068741333719452668722162067985183229670692342876321523830241441391055720270393331012876249364698564541529547943143327894588718403641405512517165098069320271203386154187615520979605105000683528885233998622790846447132350844950716645464749270401888013463811945436603516318889175596963945846513589211578330246067449709113772108686975512562489775137437585893387313621325722597716490059296607688290436508719041987188863149400902383943147121252585814892697701270568659

65537


In [7]:
falta = len(bin(personal)[2:]) % 4
print(falta)

if falta != 0:
    binpersona = (4 - falta)*'0' + bin(personal)[2:]
else:
    binpersona = bin(personal)[2:]

0


In [8]:
N = len(binpersona)//4
N

512

In [9]:
primero = binpersona[0:N]
segundo = binpersona[N:2*N]
tercero = binpersona[2*N:3*N]
cuarto = binpersona[3*N:4*N]

In [10]:
personal == int(primero+segundo+tercero+cuarto, 2)

True

In [11]:
maximo = 2**(2*N)
maximo

179769313486231590772930519078902473361797697894230657273430081157732675805500963132708477322407536021120113879871393357658789768814416622492847430639474124377767893424865485276302219601246094119453082952085005768838150682342462881473913110540827237163350510684586298239947245938479716304835356329624224137216

In [12]:
var('x')
for i in range(0, 2**N):
    c = int(bin(i)[2:] + segundo + tercero, 2)
    primeroAct = bin(int(primero, 2) - i)[2:]
    RS = int(primeroAct + cuarto, 2)
    RSinv = int(cuarto + primeroAct, 2)
    r2s2 = c - RSinv
    rs2 = r2s2 + 2*RS
    rs = sqrt(rs2)
    sol = solve(x**2 - x*rs + RS, x)
    if type(sol[0]) == Integer:
        print(sol)
        break

[9277935772435706960851542806112452521695201368671289570486443633209936969651642515965792810959618353409551798129710084457365197899124561553934898868088651, 12878666696514603471823345515110170619811871064264653197020057476597468977003373052144058183266541905850894313195966929060951959906930635651811512581640601]


In [13]:
sol

[9277935772435706960851542806112452521695201368671289570486443633209936969651642515965792810959618353409551798129710084457365197899124561553934898868088651,
 12878666696514603471823345515110170619811871064264653197020057476597468977003373052144058183266541905850894313195966929060951959906930635651811512581640601]

In [14]:
p = int(bin(sol[0])[2:] + bin(sol[1])[2:], 2)
q = int(bin(sol[1])[2:] + bin(sol[0])[2:], 2)

In [15]:
p * q == personal

True

In [16]:
phi = (p-1)*(q-1)

private_exponent = pow(public_exponent, -1, phi)

key = RSA.construct((int(personal), int(public_exponent), int(private_exponent)), consistency_check=False)

In [17]:
f = open('privateKey_Miquel_Umbert.pem','wb')
f.write(key.exportKey('PEM'))
f.close()

In [18]:
os.system('openssl rsautl -decrypt -in miquel.umbert.bosch_RSA_pseudo.enc -out AESkey.txt -inkey privateKey_Miquel_Umbert.pem')

0

In [19]:
os.system('openssl enc -d -aes-128-cbc -pbkdf2 -kfile AESkey.txt -in miquel.umbert.bosch_AES_pseudo.enc -out desencriptado.txt')

0

In [20]:
with open('desencriptado.txt', 'rb') as f:
    desencriptado = f.read()

In [21]:
tipo = filetype.guess(desencriptado)
extension = tipo.extension

In [22]:
with open('MiquelUmbert.' + extension, "wb") as binary_file:
        binary_file.write(desencriptado)